In [1]:
%matplotlib inline

In [1]:
import cv2
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mostCommon import most_common
from sklearn import preprocessing
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix


In [2]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig('' + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)

def makeModel():
    drop=0.01
    kernel=(2,2)
    LR=1e-3
    opt = Adam(learning_rate=LR, decay=LR / 20)
    # Create model
    model = Sequential()

    model.add(Conv2D(16, kernel_size=kernel, activation='relu', 
                     kernel_initializer='he_uniform', padding='same', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))

    model.add(Conv2D(16, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(drop))

    model.add(Conv2D(32, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(32, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(drop))

    model.add(Conv2D(64, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(drop))

    model.add(Conv2D(126, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(126, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(drop))

    model.add(Conv2D(256, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(256, kernel_size=kernel, activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(drop))
    
    #Flatten results
    model.add(Flatten())

    #Hidden layers of 512 neurons
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [3]:
IMAGE_WIDTH=256
IMAGE_HEIGHT=256

In [4]:
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2])
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,3)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])

In [5]:
#Label encode classes to numbers
number = preprocessing.LabelEncoder()
y=number.fit_transform(y)
#One-hot Encode y
y=to_categorical(y,3)

# Scale the pixels
X=X/255.0

In [6]:
# initialize the training data augmentation object
MODEL_PATH=os.path.join('', "Conv2d-Better")



mcp_save = ModelCheckpoint(
    MODEL_PATH,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [7]:
BATCH_SIZE=16
EPOCHS=20
LR=0.2

# Define callbacks
earlyStopping = EarlyStopping(
    monitor='loss',
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)


reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=7,
    verbose=1,
    min_delta=1e-4,
    mode='min'
)


In [8]:
l=5
ll=[-5,-4,-3,-2,-1]
patients_predictions=[]
patients_actual=[]

frame_predictions=[]
o=19
for o in range(122):
    print('---------------------------- Patient ' + str(o) + ' ------------------------------ \n')
    
    trainAug = ImageDataGenerator(
    rotation_range=10,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    )
    ll=[t+l for t in ll]
    print('Indexes: '+str(ll))
    X_test=[]
    y_test=[]
    X_train=[]
    y_train=[]
    X_test=np.take(X,ll,axis=0)
    y_test=np.take(y,ll,axis=0)
    X_train=np.delete(X,ll,axis=0)
    y_train=np.delete(y,ll,axis=0)
    

    # Creating array test splits for models
    model=makeModel()
    
    
    print('Training Model...')
    history=model.fit(
    trainAug.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=10
    )
    
    

    print('getting predictions of each model...')
    prediction=model.predict_classes(X_test)
    patients_predictions.append(most_common(prediction.tolist()))
    patients_actual.append(np.argmax(y_test[0]))
    
    for i in prediction:
        frame_predictions.append(i)
    keras.backend.clear_session()
    
    
    
print('All Done :)')


---------------------------- Patient 0 ------------------------------ 

Indexes: [0, 1, 2, 3, 4]
Training Model...
Epoch 1/10
37/37 [==============================] - 15s 255ms/step - loss: 1.4655 - accuracy: 0.5212
Epoch 2/10
37/37 [==============================] - 10s 257ms/step - loss: 0.7350 - accuracy: 0.6995
Epoch 3/10
37/37 [==============================] - 9s 234ms/step - loss: 0.6453 - accuracy: 0.7504
Epoch 4/10
37/37 [==============================] - 14s 389ms/step - loss: 0.5772 - accuracy: 0.7606
Epoch 5/10
37/37 [==============================] - 9s 235ms/step - loss: 0.4969 - accuracy: 0.7912
Epoch 6/10
37/37 [==============================] - 9s 235ms/step - loss: 0.4578 - accuracy: 0.8132
Epoch 7/10
37/37 [==============================] - 9s 231ms/step - loss: 0.5141 - accuracy: 0.7956
Epoch 8/10
37/37 [==============================] - 9s 249ms/step - loss: 0.4115 - accuracy: 0.8421
Epoch 9/10
37/37 [==============================] - 9s 236ms/step - loss: 0.4493 -

C:\Users\RS3\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


NameError: name 'keras' is not defined

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'],
           loc='upper left')
plt.show()

In [ ]:
#Patient Confusion Matrix

In [ ]:
#Frames Confusion Matrix